# Logrank optimization

This notebook presents a proposal for optimizing logrank computation time.

In [82]:
from time import perf_counter
import pandas as pd
import numpy as np

import old_kaplan_meier
import new_kaplan_meier
import rulekit_kaplan_meier


old_impl = old_kaplan_meier.KaplanMeierEstimator
rulekit_impl = rulekit_kaplan_meier.KaplanMeierEstimator
new_impl = new_kaplan_meier.PrecalculatedKaplanMeierEstimator


df_train: pd.DataFrame = pd.read_parquet("./train.parquet")
X_train, y_train = df_train.drop("survival_status", axis=1), df_train["survival_status"]

X_train.describe()

,ici,thick,age,survival_time
count,143.000000,143.000000,143.000000,143.000000
mean,1.664336,2.565245,52.223776,2167.363636
std,0.777718,2.269986,16.812730,1040.141191
min,0.000000,0.160000,4.000000,30.000000
25%,1.000000,0.970000,41.500000,1613.000000
50%,2.000000,1.780000,54.000000,2028.000000
75%,2.000000,3.540000,64.000000,3054.500000
max,3.000000,12.880000,95.000000,4479.000000


In [83]:
from decision_rules.conditions import ElementaryCondition

X_np = X_train.sort_values(by='survival_time', ascending=True).to_numpy()
y_np = y_train.to_numpy().astype(int).astype(str)
survival_time_idx = X_train.columns.get_loc("survival_time")

c = ElementaryCondition(column_index=X_train.columns.get_loc("age"), right=54)
print("Condition:", c.to_string(X_train.columns))
cov_mask = c.covered_mask(X_np)
uncov_mask = c.uncovered_mask(X_np)
print(
    f"Covered count: {np.where(cov_mask == 1)[0].shape[0]}  "
    f"Uncovered count: {len(X_np) - np.where(cov_mask == 1)[0].shape[0]}"
)

Condition: age < 54.00
Covered count: 66  Uncovered count: 77


## Compare results

### Current implementation 

Slow 🐌

In [84]:
old_impl.log_rank(
    X_np[:, survival_time_idx], y_np, cov_mask, uncov_mask
)

0.9581253739626425

### Implementation based on [current RuleKit code](https://github.com/adaa-polsl/RuleKit/blob/master/adaa.analytics.rules/src/main/java/adaa/analytics/rules/logic/quality/LogRank.java) after latest optimization 

Still quite
slow though and result somehow changed (might be me rewriting it from Java to Python)

In [85]:
rulekit_impl.log_rank(
    X_np[:, survival_time_idx], y_np, cov_mask, uncov_mask
)

0.9888123174388482

### New implementation 

Is fast and gives the same results as current one. 🏎️💨

In [86]:
new_impl.log_rank(
    X_np[:, survival_time_idx], y_np, cov_mask, uncov_mask, skip_sorting=True
)

0.9581253739626425

## Compare performance


In [87]:
N = 1000


### Current implementation

In [88]:
start = perf_counter()
for _ in range(N):
    old_impl.log_rank(
        X_np[:, survival_time_idx],
        y_np,
        cov_mask,
        uncov_mask,
    )
old_impl_time = perf_counter() - start
print(old_impl_time, 'seconds')

3.8970611999975517 seconds



### Rulekit based implementation 

Faster but not significantly 

In [89]:
start = perf_counter()
for _ in range(N):
    rulekit_impl.log_rank(
        X_np[:, survival_time_idx],
        y_np,
        cov_mask,
        uncov_mask,
    )
rulekit_impl_time = perf_counter() - start
print(rulekit_impl_time, "seconds")

3.5177531000226736 seconds


### Proposed implementation

In [90]:
start = perf_counter()
for _ in range(N):
    new_impl.log_rank(
        X_np[:, survival_time_idx],
        y_np,
        cov_mask,
        uncov_mask,
        skip_sorting=True
    )
new_impl_time = perf_counter() - start
print(new_impl_time, "seconds")

0.5014416000049096 seconds


In [91]:
print(
    "Proposed implemetation is: "
    f"{round(old_impl_time / new_impl_time, 2)}"
    " times faster than the current one \nand "
    f"{round(rulekit_impl_time / new_impl_time, 2)}"
    " times faster than the one based on RuleKit code"
)

Proposed implemetation is: 7.77 times faster than the current one 
and 7.02 times faster than the one based on RuleKit code


### Execution times comparison on different dataset sizes

In [118]:
times = []

np.errstate(all='ignore')

for i in range(1, 10):
    df: pd.DataFrame = pd.read_parquet("./train.parquet").sort_values(
        by="survival_time", ascending=True
    )
    df = pd.concat([df_train] * i)
    X, y = df.drop("survival_status", axis=1), df["survival_status"]
    survival_time_idx = X.columns.get_loc("survival_time")
    surv_times = np.arange(X.shape[0])

    X = X.to_numpy()
    y = y.to_numpy().astype(int).astype(str)
    c = ElementaryCondition(column_index=X_train.columns.get_loc("age"), right=54)
    cov_mask = c.covered_mask(X)
    uncov_mask = c.uncovered_mask(X)

    start = perf_counter()
    for _ in range(100):
        new_impl.log_rank(
            surv_times, y, cov_mask, uncov_mask, skip_sorting=True
        )
    new_impl_time = perf_counter() - start

    start = perf_counter()
    for _ in range(100):
        old_impl.log_rank(
            surv_times,
            y,
            cov_mask,
            uncov_mask,
        )
    old_impl_time = perf_counter() - start

    times.append({"n": X.shape[0], "after": new_impl_time, "before": old_impl_time})

times = pd.DataFrame(times)
times

,n,after,before
0,143,0.055157,0.404043
1,286,0.088375,0.475430
2,429,0.131518,0.510039
3,572,0.177960,0.579416
4,715,0.319667,0.637883
5,858,0.254716,0.696119
6,1001,0.294040,0.737754
7,1144,0.398587,0.800243
8,1287,0.376764,0.847568


In [120]:
import plotly.express as px

px.line(times, x='n', y=['before', 'after'], title='Execution time based on dataset size')